In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


# DATA PREPROCESSING TRAIN

### LOAD DATA

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"harismohomed","key":"defb1a6bd72ffb88a8c9e259faae5a31"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install kaggle
!kaggle datasets download -d odins0n/amex-parquet

100%|██████████████████████████████████████▉| 8.64G/8.65G [01:02<00:00, 115MB/s]
100%|███████████████████████████████████████| 8.65G/8.65G [01:02<00:00, 149MB/s]


In [4]:
!unzip "/kaggle/working/amex-parquet.zip"

Archive:  /kaggle/working/amex-parquet.zip
  inflating: test_data.parquet       
  inflating: train_data.parquet      


In [5]:
!rm "/kaggle/working/amex-parquet.zip"
gc.collect()

124

In [6]:
DATA_PATH = "/kaggle/working/train_data.parquet"
TEST_DATA_PATH = "/kaggle/working/test_data.parquet"
SAMPLE_SUBMISSION_PATH = "/kaggle/input/amex-default-prediction/sample_submission.csv"

### READ DATA

In [7]:
data_loaded = pd.read_parquet(DATA_PATH)

In [8]:
# Let's Keep Latest For Each Customer
data_sorted=data_loaded.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()
data_sorted = data_sorted.drop(['S_2'],axis=1)
del data_loaded
gc.collect()

23

In [9]:
data_sorted.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,NaN,NaN,...,NaN,NaN,0.007186,0.004234,0.005086,NaN,0.005810,0.002970,0.008533,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,NaN,0.060646,...,NaN,NaN,0.002980,0.007479,0.007870,NaN,0.003284,0.003169,0.008514,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.880875,0.009704,0.004284,0.812649,0.006450,NaN,0.003796,0.007196,NaN,NaN,...,NaN,NaN,0.007383,0.006623,0.000964,NaN,0.002202,0.000834,0.003444,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.621776,0.001083,0.012564,1.006183,0.007829,0.287766,0.004532,0.009937,NaN,0.046104,...,NaN,NaN,0.002704,0.006184,0.001899,NaN,0.008183,0.005560,0.002983,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.871900,0.005573,0.007679,0.815746,0.001247,NaN,0.000231,0.005528,NaN,0.044671,...,NaN,NaN,0.002974,0.004162,0.005764,NaN,0.008154,0.006944,0.000905,0


In [10]:
data_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 458913 entries, 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a to fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681
Columns: 189 entries, P_2 to target
dtypes: float32(185), int64(2), object(2)
memory usage: 341.4+ MB


In [11]:
data_sorted.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,455944.000000,4.589130e+05,458913.000000,4.588820e+05,4.589130e+05,373943.000000,4.588820e+05,4.588820e+05,59910.000000,324591.000000,...,1.639500e+04,1.639500e+04,4.560830e+05,4.589130e+05,4.560830e+05,80315.000000,4.560830e+05,4.589130e+05,4.560830e+05,458913.000000
mean,0.635249,2.014896e-01,0.139027,5.916128e-01,1.073958e-01,0.229064,9.948801e-02,1.548291e-01,0.177988,0.163768,...,1.414993e-02,1.590173e-01,1.811025e-01,2.893126e-02,1.665783e-01,0.394761,1.809953e-01,5.201584e-02,6.379602e-02,0.258934
std,0.267547,4.021564e-01,0.231266,4.121456e-01,2.786386e-01,0.198845,3.040705e-01,2.550615e-01,0.253631,0.227059,...,9.529648e-02,2.610814e-01,3.809093e-01,1.528808e-01,3.501717e-01,0.241254,3.808317e-01,1.820323e-01,1.970930e-01,0.438050
min,-0.458955,7.406592e-08,-3.571172,2.024839e-07,5.646122e-08,-0.508883,3.998730e-08,7.574315e-08,-0.000240,0.000002,...,3.704722e-07,1.703873e-07,2.207831e-08,7.897955e-09,9.490088e-08,-0.011438,1.653580e-08,1.995319e-08,2.811854e-09,0.000000
25%,0.449332,4.782223e-03,0.008984,7.091162e-02,2.957326e-03,0.127512,3.008724e-03,5.483745e-03,0.031770,0.043333,...,2.518881e-03,3.482435e-03,3.045516e-03,2.551175e-03,3.027868e-03,0.200727,3.032092e-03,2.757897e-03,3.031604e-03,0.000000
50%,0.682559,9.559730e-03,0.033219,8.134054e-01,5.920879e-03,0.164910,6.010615e-03,1.082996e-02,0.108293,0.092436,...,5.052533e-03,7.028427e-03,6.078700e-03,5.120597e-03,6.050620e-03,0.386107,6.065281e-03,5.501410e-03,6.072661e-03,0.000000
75%,0.862186,2.665587e-01,0.153634,1.001956e+00,8.891468e-03,0.264784,8.998210e-03,2.106605e-01,0.230394,0.195026,...,7.578313e-03,5.013750e-01,9.101306e-03,7.674505e-03,9.101262e-03,0.573958,9.097647e-03,8.251762e-03,9.101819e-03,1.000000
max,1.009998,5.001550e+00,1.324059,1.010000e+00,2.759738e+00,3.993525,8.988807e+00,1.596532e+00,4.186139,9.185312,...,1.009950e+00,2.505057e+00,1.010000e+00,1.010000e+00,1.339910e+00,2.229182,1.010000e+00,1.343326e+00,4.827630e+00,1.000000


In [12]:
gc.collect()

23

### FINDINGS

In [13]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
target_cols = ['target']
numerical_cols = [num_col for num_col in data_sorted.columns if num_col not in cat_cols+target_cols ]


print("CAT COLS : ",str(cat_cols))
print("NUM COLS : ",str(numerical_cols))
print("TARGET COLS : ",str(target_cols))

CAT COLS :  ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
NUM COLS :  ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_8

### CHECK NAN AND DROP LARGELY MISSING COLUMNS

In [14]:
total_rows = len(data_sorted)
threshold_to_remove_nan=total_rows * 0.75

CHECK NAN LARGE THAN THRES HOLD

In [15]:
null_count = pd.DataFrame(data_sorted.isnull().sum(axis=0))
largely_missing_columns = list(null_count.loc[null_count[0]>threshold_to_remove_nan].index)
largely_missing_columns

['D_42',
 'D_49',
 'D_66',
 'D_73',
 'D_76',
 'R_9',
 'B_29',
 'D_87',
 'D_88',
 'D_106',
 'R_26',
 'D_108',
 'D_110',
 'D_111',
 'B_39',
 'B_42',
 'D_132',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_142']

In [16]:
data_sorted = data_sorted.drop(largely_missing_columns,axis=1)

Update CAT,NUM List

In [17]:
numerical_cols = [col for col in numerical_cols if col not in largely_missing_columns]
cat_cols = [col for col in cat_cols if col not in largely_missing_columns]

In [18]:
gc.collect()

138

In [19]:
data_sorted.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
count,455944.000000,4.589130e+05,458913.000000,4.588820e+05,4.589130e+05,373943.000000,4.588820e+05,4.588820e+05,324591.000000,4.366180e+05,...,4.560830e+05,4.589130e+05,4.589130e+05,4.560830e+05,4.589130e+05,4.560830e+05,4.560830e+05,4.589130e+05,4.560830e+05,458913.000000
mean,0.635249,2.014896e-01,0.139027,5.916128e-01,1.073958e-01,0.229064,9.948801e-02,1.548291e-01,0.163768,1.386199e-01,...,1.167025e-01,5.146633e-02,5.660660e-03,1.811025e-01,2.893126e-02,1.665783e-01,1.809953e-01,5.201584e-02,6.379602e-02,0.258934
std,0.267547,4.021564e-01,0.231266,4.121456e-01,2.786386e-01,0.198845,3.040705e-01,2.550615e-01,0.227059,2.471693e-01,...,3.122695e-01,1.828866e-01,2.588725e-02,3.809093e-01,1.528808e-01,3.501717e-01,3.808317e-01,1.820323e-01,1.970930e-01,0.438050
min,-0.458955,7.406592e-08,-3.571172,2.024839e-07,5.646122e-08,-0.508883,3.998730e-08,7.574315e-08,0.000002,1.031370e-08,...,5.461693e-09,1.243154e-09,4.549750e-08,2.207831e-08,7.897955e-09,9.490088e-08,1.653580e-08,1.995319e-08,2.811854e-09,0.000000
25%,0.449332,4.782223e-03,0.008984,7.091162e-02,2.957326e-03,0.127512,3.008724e-03,5.483745e-03,0.043333,3.995819e-03,...,2.823318e-03,2.754521e-03,2.504165e-03,3.045516e-03,2.551175e-03,3.027868e-03,3.032092e-03,2.757897e-03,3.031604e-03,0.000000
50%,0.682559,9.559730e-03,0.033219,8.134054e-01,5.920879e-03,0.164910,6.010615e-03,1.082996e-02,0.092436,8.030155e-03,...,5.645906e-03,5.517987e-03,4.998674e-03,6.078700e-03,5.120597e-03,6.050620e-03,6.065281e-03,5.501410e-03,6.072661e-03,0.000000
75%,0.862186,2.665587e-01,0.153634,1.001956e+00,8.891468e-03,0.264784,8.998210e-03,2.106605e-01,0.195026,1.341534e-01,...,8.457952e-03,8.264314e-03,7.498714e-03,9.101306e-03,7.674505e-03,9.101262e-03,9.097647e-03,8.251762e-03,9.101819e-03,1.000000
max,1.009998,5.001550e+00,1.324059,1.010000e+00,2.759738e+00,3.993525,8.988807e+00,1.596532e+00,9.185312,4.005466e+00,...,1.491624e+00,1.509998e+00,1.009960e+00,1.010000e+00,1.010000e+00,1.339910e+00,1.010000e+00,1.343326e+00,4.827630e+00,1.000000


### IMPUTING

In [20]:
from sklearn.impute import SimpleImputer
imp_numerical = SimpleImputer(missing_values=np.nan, strategy='median')
imp_numerical.fit(data_sorted[numerical_cols])
data_sorted[numerical_cols] = imp_numerical.transform(data_sorted[numerical_cols])

In [21]:
for col in cat_cols:
    data_sorted[col] =  data_sorted[col].fillna(data_sorted[col].mode()[0])

In [22]:
data_sorted.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,0.092436,0.003258,...,0.005702,0.006210,0.002715,0.007186,0.004234,0.005086,0.005810,0.002970,0.008533,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,0.060646,0.008781,...,0.001928,0.002996,0.001701,0.002980,0.007479,0.007870,0.003284,0.003169,0.008514,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.880875,0.009704,0.004284,0.812649,0.006450,0.164910,0.003796,0.007196,0.092436,0.000628,...,0.003470,0.009881,0.007691,0.007383,0.006623,0.000964,0.002202,0.000834,0.003444,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.621776,0.001083,0.012564,1.006183,0.007829,0.287766,0.004532,0.009937,0.046104,0.007792,...,0.004576,0.001789,0.005140,0.002704,0.006184,0.001899,0.008183,0.005560,0.002983,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.871900,0.005573,0.007679,0.815746,0.001247,0.164910,0.000231,0.005528,0.044671,0.002436,...,0.008897,0.005045,0.003706,0.002974,0.004162,0.005764,0.008154,0.006944,0.000905,0


In [23]:
null_count = pd.DataFrame(data_sorted.isnull().sum(axis=0))
list(null_count.loc[null_count[0]>0].index)

[]

In [24]:
gc.collect()

23

In [25]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
data_sorted[cat_cols] = enc.fit_transform(data_sorted[cat_cols])

In [26]:
data_sorted.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,0.092436,0.003258,...,0.005702,0.006210,0.002715,0.007186,0.004234,0.005086,0.005810,0.002970,0.008533,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,0.060646,0.008781,...,0.001928,0.002996,0.001701,0.002980,0.007479,0.007870,0.003284,0.003169,0.008514,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.880875,0.009704,0.004284,0.812649,0.006450,0.164910,0.003796,0.007196,0.092436,0.000628,...,0.003470,0.009881,0.007691,0.007383,0.006623,0.000964,0.002202,0.000834,0.003444,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.621776,0.001083,0.012564,1.006183,0.007829,0.287766,0.004532,0.009937,0.046104,0.007792,...,0.004576,0.001789,0.005140,0.002704,0.006184,0.001899,0.008183,0.005560,0.002983,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.871900,0.005573,0.007679,0.815746,0.001247,0.164910,0.000231,0.005528,0.044671,0.002436,...,0.008897,0.005045,0.003706,0.002974,0.004162,0.005764,0.008154,0.006944,0.000905,0


In [27]:
corr = data_sorted.corr()
corelated_var= set()

for i in range(len(corr.columns)):
    for j in range(i):
        if(corr.iloc[i, j] > 0.9):
            colname = corr.columns[i]
            corelated_var.add(colname)

In [28]:
corelated_var

{'B_11',
 'B_13',
 'B_15',
 'B_23',
 'B_33',
 'B_37',
 'D_104',
 'D_119',
 'D_141',
 'D_143',
 'D_74',
 'D_75',
 'D_77',
 'S_24',
 'S_7'}

In [29]:
data_sorted = data_sorted.drop(corelated_var,axis=1)

In [30]:
data_sorted.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,0.092436,0.003258,...,0.003973,0.004186,0.005702,0.006210,0.002715,0.007186,0.004234,0.002970,0.008533,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,0.060646,0.008781,...,0.002943,0.002202,0.001928,0.002996,0.001701,0.002980,0.007479,0.003169,0.008514,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.880875,0.009704,0.004284,0.812649,0.006450,0.164910,0.003796,0.007196,0.092436,0.000628,...,0.004133,0.002654,0.003470,0.009881,0.007691,0.007383,0.006623,0.000834,0.003444,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.621776,0.001083,0.012564,1.006183,0.007829,0.287766,0.004532,0.009937,0.046104,0.007792,...,0.004254,0.000060,0.004576,0.001789,0.005140,0.002704,0.006184,0.005560,0.002983,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.871900,0.005573,0.007679,0.815746,0.001247,0.164910,0.000231,0.005528,0.044671,0.002436,...,0.001068,1.006119,0.008897,0.005045,0.003706,0.002974,0.004162,0.006944,0.000905,0


In [31]:
gc.collect()

23

# DATA PREPROCESSING TEST

In [32]:
test_data_loaded = pd.read_parquet(TEST_DATA_PATH)

In [33]:
# Let's Keep Latest For Each Customer
test_data_sorted=test_data_loaded.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()
test_data_sorted = test_data_sorted.drop(['S_2'],axis=1)
del test_data_loaded
gc.collect()

0

In [34]:
test_data_sorted = test_data_sorted.drop(largely_missing_columns,axis=1)

In [35]:
from sklearn.impute import SimpleImputer
imp_numerical = SimpleImputer(missing_values=np.nan, strategy='median')
imp_numerical.fit(test_data_sorted[numerical_cols])
test_data_sorted[numerical_cols] = imp_numerical.transform(test_data_sorted[numerical_cols])

In [36]:
for col in cat_cols:
    test_data_sorted[col] =  test_data_sorted[col].fillna(test_data_sorted[col].mode()[0])

In [37]:
null_count = pd.DataFrame(test_data_sorted.isnull().sum(axis=0))
list(null_count.loc[null_count[0]>threshold_to_remove_nan].index)

[]

In [38]:
gc.collect()

23

In [39]:
from sklearn.preprocessing import OrdinalEncoder
test_data_sorted[cat_cols] = enc.fit_transform(test_data_sorted[cat_cols])

In [40]:
test_data_sorted = test_data_sorted.drop(corelated_var,axis=1)

In [41]:
test_data_sorted.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_144,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.568930,0.121385,0.010779,1.009347,0.006923,0.149413,0.000396,0.003576,0.007398,0.006787,...,0.009961,0.006789,0.004902,0.000975,0.006273,0.008816,0.005912,0.001250,0.003690,0.003219
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.841177,0.126475,0.016562,1.009245,0.009715,0.112195,0.006192,0.011386,0.096836,0.004234,...,0.001249,0.007340,0.001252,0.007632,0.002767,0.008789,0.004344,0.000866,0.000247,0.007780
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.697522,0.002724,0.001484,0.810072,0.002620,0.166165,0.004888,0.015938,0.105303,0.003381,...,0.009323,0.007062,1.003890,0.001152,0.002045,0.001852,1.001246,0.008894,0.457819,0.092041
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.513186,0.324828,0.149511,0.205678,0.002278,0.181200,0.005813,0.498516,0.211615,0.258627,...,0.009126,0.002427,0.009438,0.002776,0.009377,0.003622,1.008246,0.003753,0.500924,0.183020
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.254478,0.768016,0.563603,0.038021,0.503154,0.168317,0.009480,0.830857,0.071884,0.375839,...,1.008068,0.004532,0.003340,0.001000,0.007940,0.009232,0.006623,0.001140,0.001558,0.000525


In [42]:
gc.collect()

92

In [43]:
len(test_data_sorted)

924621

# TRAIN MODEL

In [44]:
all_feature_columns = list(data_sorted.columns)
all_feature_columns.remove('target')
X = data_sorted[all_feature_columns]
y = data_sorted['target']

In [45]:
print(X.shape)
print(y.shape)

(458913, 150)
(458913,)


In [46]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(X, y, test_size=0.01, random_state=42, stratify=y)

In [47]:
gc.collect()

115

In [62]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier, early_stopping
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

model_1 = CatBoostClassifier()
model_2 = XGBClassifier()
model_3 = LGBMClassifier()

model = VotingClassifier(verbose=True,
    estimators=[('cat', model_1), ('xgb', model_2), ('lgbm', model_3)], voting='soft')

model.fit(x_train,y_train) 

Learning rate set to 0.140485
0:	learn: 0.5403585	total: 296ms	remaining: 4m 55s
1:	learn: 0.4375006	total: 599ms	remaining: 4m 58s
2:	learn: 0.3851822	total: 841ms	remaining: 4m 39s
3:	learn: 0.3445936	total: 1.09s	remaining: 4m 31s
4:	learn: 0.3214256	total: 1.32s	remaining: 4m 23s
5:	learn: 0.3073773	total: 1.55s	remaining: 4m 17s
6:	learn: 0.2922853	total: 1.82s	remaining: 4m 18s
7:	learn: 0.2840123	total: 2.02s	remaining: 4m 11s
8:	learn: 0.2764764	total: 2.25s	remaining: 4m 7s
9:	learn: 0.2693439	total: 2.52s	remaining: 4m 9s
10:	learn: 0.2654181	total: 2.7s	remaining: 4m 2s
11:	learn: 0.2617659	total: 2.9s	remaining: 3m 58s
12:	learn: 0.2576361	total: 3.15s	remaining: 3m 59s
13:	learn: 0.2549841	total: 3.36s	remaining: 3m 56s
14:	learn: 0.2526269	total: 3.62s	remaining: 3m 57s
15:	learn: 0.2509185	total: 3.86s	remaining: 3m 57s
16:	learn: 0.2493700	total: 4.05s	remaining: 3m 54s
17:	learn: 0.2478574	total: 4.33s	remaining: 3m 56s
18:	learn: 0.2462721	total: 4.58s	remaining: 3m 5

VotingClassifier(estimators=[('cat',
                              <catboost.core.CatBoostClassifier object at 0x7efa3a7f3d10>),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type...
                                            interaction_constraints=None,
                                            learning_rate=None, max_b

In [63]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb')) 

In [64]:
y_pred = model.predict_proba(x_val)

In [65]:
y_pred_df = pd.DataFrame(y_pred[:,1],columns=["prediction"])
y_pred_df["customer_ID"]= x_val.index
y_pred_df.set_index("customer_ID",inplace=True)
y_pred_df

,prediction
customer_ID,
e7a323976e71c8def00040ba72577960db31b0847056ddec06a76725efe04574,0.000729
ac3d61c3dae86984d77b50c8552ec99eea7fc2cedbf16541c16a3934189ec804,0.000741
6d224f602013cf4013c7bd20fd8c0a60847aaf08e739eef24adc995d56d850f8,0.848699
60b2f51f9f7daebfc9132ec96ea4819c1aa95910a46b42a8374388d99dd608d3,0.973107
a7f5780dafc6d3095888db35ef907aedbd1a230e4ddd657e77333b4df211346f,0.000937
...,...
e468b316a006a1b14f0515741da78804583428e709a794f46ce57a4361b92a8a,0.763200
9013a31e4ea1a67440e40de27acb4e3105b26017dbeb3560938e20dcbcb9edea,0.991804
f5b826ebcfadfc4ab301daac5e1fd901104157f05bfe18ee357727f9cd3efccb,0.824141


In [66]:
y_val_df = pd.DataFrame(y_val)
y_val_df

,target
customer_ID,
e7a323976e71c8def00040ba72577960db31b0847056ddec06a76725efe04574,0
ac3d61c3dae86984d77b50c8552ec99eea7fc2cedbf16541c16a3934189ec804,0
6d224f602013cf4013c7bd20fd8c0a60847aaf08e739eef24adc995d56d850f8,1
60b2f51f9f7daebfc9132ec96ea4819c1aa95910a46b42a8374388d99dd608d3,1
a7f5780dafc6d3095888db35ef907aedbd1a230e4ddd657e77333b4df211346f,0
...,...
e468b316a006a1b14f0515741da78804583428e709a794f46ce57a4361b92a8a,0
9013a31e4ea1a67440e40de27acb4e3105b26017dbeb3560938e20dcbcb9edea,1
f5b826ebcfadfc4ab301daac5e1fd901104157f05bfe18ee357727f9cd3efccb,1


# EVAL METRICS

In [67]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        print(df)
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [68]:
amex_metric(y_val_df,y_pred_df)

                                                    target  prediction
customer_ID                                                           
a327d896296786abe8aa3c57fbb8f2aeb4dc0e794c8304b...       1    0.999014
05ee43092ac5b239eae4189334a8c6f002b66e091574b69...       1    0.998866
295172c006324f49e239cd0cbc9caa7804aa3f6f96db43d...       1    0.998804
e3a619a63a3bd660c7723bc9a0b784c221024d7c0a497bf...       1    0.998781
7b8b921cfdd7925b96d98dce9c3372995797ed5139abaea...       1    0.998736
...                                                    ...         ...
fa2f2fe62054ffb54cb1b3e067e88b2e0a1b11d5d88fcd7...       0    0.000474
d4bdf5622ee67443b9b3b57e0b66aea8348e6e1f1437ac7...       0    0.000473
448572a16a821b06683d2d29e680db36034604496b91ab2...       0    0.000469
3da41e69ef224fe650a615d90eb86a3d46c07994b771a26...       0    0.000429
2226c8c7192ae437dadfc7a9c93122c9294fd01ce809978...       0    0.000375

[4590 rows x 2 columns]


0.79476079413974

# PREDICT FOR TEST DATA

In [69]:
# CHUNK_SIZE = 10000
# y_test_pred = []
# for chunk_num in range(len(test_data_sorted) // CHUNK_SIZE + 1):
#     start_index = chunk_num*CHUNK_SIZE
#     end_index = min(chunk_num*CHUNK_SIZE + CHUNK_SIZE, len(test_data_sorted))
#     chunk = test_data_sorted[start_index:end_index]
    
#     preds = model.predict_proba(test_data_sorted)
#     y_test_pred = y_test_pred + list(preds[:,1])
#     print("\nChunk No : ",chunk_num)
#     print("Completed Predictions : ",len(y_test_pred))
#     print("Completed : ",str(end_index*100/len(test_data_sorted))," % ")


In [70]:
def chunker(seq, size):
        for pos in range(0, len(seq), size):
            yield seq.iloc[pos:pos + size] 


In [ ]:
res=[]
chunk_size=1
print(len(test_data_sorted)/chunk_size)
for idf in chunker(test_data_sorted, chunk_size):
    if len(res)%1000==0:
        print(len(res))
    iy_test_pred = model.predict_proba(idf)
    res.append(iy_test_pred[:,1])

924621.0
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000


In [ ]:
print(res[0][0])  

In [ ]:
y_test_pred=[]
for i in range(len(res)):
    y_test_pred.append(res[i][0]) 
    
y_test_pred[0:10]

In [ ]:
sample_dataset = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
output = pd.DataFrame({'customer_ID': sample_dataset.customer_ID, 'prediction': y_test_pred})
output

In [ ]:
output.to_csv('submission-1.csv', index=False)

In [ ]:
in1 = pd.read_csv('submission-1.csv')

In [ ]:
in1.head()